# RFM Analysis

In [2]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [10]:
df = pd.read_excel("D:\document\data_science\Mini Project dan final Project\RFM Analysis.xlsx")

In [4]:
df.shape

(2155, 4)

In [5]:
df.head()

,OrderID,OrderDate,CustomerID,Monetory
0,10248,1996-07-04 00:00:00.000,VINET,168.0
1,10248,1996-07-04 00:00:00.000,VINET,98.0
2,10248,1996-07-04 00:00:00.000,VINET,174.0
3,10249,1996-07-05 00:00:00.000,TOMSP,167.4
4,10249,1996-07-05 00:00:00.000,TOMSP,1696.0


In [12]:
df = df[df["OrderDate"] < "1998-01-01"]

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1464 entries, 0 to 1463
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   OrderID     1464 non-null   int64         
 1   OrderDate   1464 non-null   datetime64[ns]
 2   CustomerID  1464 non-null   object        
 3   Monetory    1464 non-null   float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 57.2+ KB


In [18]:
df['OrderDate'] = pd.to_datetime(df['OrderDate'])

# FRM table

In [7]:
import datetime as dt
NOW = dt.datetime(1998,1,1)

In [8]:
df['OrderDate'] = pd.to_datetime(df['OrderDate'])

In [44]:
rfmtable = df.groupby('CustomerID').agg({'OrderDate': lambda x: (NOW - x.max()).days,
                                        'OrderID': lambda x: len(x),
                                        'Monetory': lambda x: x.sum()})

rfmtable["OrderDate"] = rfmtable['OrderDate'].astype(int)
rfmtable.rename(columns={'OrderDate': 'recency',
                        'OrderID': 'frequency'}, inplace=True)
                    

# RFM quartiles

In [45]:
quntiles = rfmtable.quantile(q=[0.25,0.5,0.75])

In [46]:
quntiles

,recency,frequency,Monetory
0.25,14.0,7.0,1655.4475
0.50,43.0,15.0,5616.2150
0.75,106.5,22.0,11950.8250


In [47]:
quntiles = quntiles.to_dict()

In [48]:
quntiles

{'recency': {0.25: 14.0, 0.5: 43.0, 0.75: 106.5},
 'frequency': {0.25: 7.0, 0.5: 15.0, 0.75: 22.0},
 'Monetory': {0.25: 1655.4475, 0.5: 5616.215, 0.75: 11950.825}}

# RFM segmentation table

In [49]:
rfmsegmentation = rfmtable

In [50]:
rfmsegmentation

,recency,frequency,Monetory
CustomerID,,,
ALFKI,80,6,2294.00
ANATR,34,6,888.55
ANTON,98,15,6855.35
AROUT,8,23,7968.00
BERGS,16,36,18857.60
...,...,...,...
WARTH,15,35,16317.10
WELLI,2,13,5235.70
WHITC,49,27,13794.55


In [42]:
# Argument (x = value, p = recency, monetory, frequncy,k = quantile dict)
def Rclass(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.5]:
        return 2
    elif x <= d[p][0.75]:
        return 3
    else:
        return 4

# Argument (x = value, p = recency, monetory, frequncy,k = quantile dict)
def FMclass(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.5]:
        return 3
    elif x <= d[p][0.75]:
        return 2
    else:
        return 1

In [56]:
rfmsegmentation['R_score'] = rfmsegmentation['recency'].apply(Rclass, args=('recency',quntiles))
rfmsegmentation['F_score'] = rfmsegmentation['frequency'].apply(FMclass, args=('frequency',quntiles))
rfmsegmentation['M_score'] = rfmsegmentation['Monetory'].apply(Rclass, args=('Monetory',quntiles))

In [58]:
rfmsegmentation['RFMClass'] = rfmsegmentation.R_score.map(str) \
                            + rfmsegmentation.F_score.map(str) \
                            + rfmsegmentation.M_score.map(str)

In [59]:
rfmsegmentation.head()

,recency,frequency,Monetory,R_score,F_score,M_scpre,M_score,RFMClass
CustomerID,,,,,,,,
ALFKI,80,6,2294.00,3,4,2,2,342
ANATR,34,6,888.55,2,4,1,1,241
ANTON,98,15,6855.35,3,3,3,3,333
AROUT,8,23,7968.00,1,1,3,3,113
BERGS,16,36,18857.60,2,1,4,4,214
